# Eyeriss - V2

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py

## Initialize matrices

Dense and sparse examples are provided. The active example is selected below...


In [ ]:
ies = {}
fs = {}
o_verifies = {}

instance = "dense"
ies[instance] = Tensor.fromUncompressed(["C", "W"], 
                                       [ [1, 2, 5, 3, 4, 9, 8],
                                         [8, 1, 3, 4, 3, 5, 9] ])

fs[instance] = Tensor.fromUncompressed(["C", "S", "M"],
                                       [[[ 1, 2, 4, 3, 4, 9, 1],
                                         [ 8, 1, 5, 3, 8, 6, 2],
                                         [ 7, 6, 5, 7, 4, 3, 4]],
                                        [[ 1, 9, 5, 3, 2, 6, 9],
                                         [ 4, 4, 9, 8, 1, 4, 8],
                                         [ 2, 4, 1, 6, 1, 9, 1]]])

o_verifies[instance] = Tensor.fromUncompressed(["M", "Q"],
                                               [[70, 84, 82, 124, 173],
                                                [122, 64, 92, 132, 148],
                                                [91, 84, 109, 129, 170],
                                                [94, 93, 111, 150, 198],
                                                [60, 69, 73, 96, 140],
                                                [115, 111, 136, 159, 233],
                                                [108, 61, 89, 112, 130]])


instance = "sparse"
ies[instance] = Tensor.fromUncompressed(["C", "W"], 
                                       [ [1, 2, 0, 3, 4, 0, 0],
                                         [2, 0, 3, 0, 0, 5, 0] ])

fs[instance] = Tensor.fromUncompressed(["C", "S", "M"],
                                       [[[ 1, 2, 0, 3, 4, 0, 0],
                                         [ 0, 1, 0, 0, 8, 0, 0],
                                         [ 0, 0, 0, 0, 0, 0, 0]],
                                        [[ 1, 0, 0, 3, 0, 0, 0],
                                         [ 0, 0, 0, 0, 0, 0, 0],
                                         [ 2, 4, 0, 6, 0, 9, 1]]])

o_verifies[instance] = Tensor.fromUncompressed(["M", "Q"],
                                               [[9, 2, 3, 13, 4, 5],
                                                [16, 4, 3, 30, 8, 0],
                                                [0, 0, 0, 0, 0, 0],
                                                [27, 6, 9, 39, 12, 15],
                                                [20, 8, 24, 44, 16, 0],
                                                [27, 0, 0, 45, 0, 0],
                                                [3, 0, 0, 5, 0, 0]])





# Number of weights in filter (should be computed)
S = 3
# Number of output activations (should be computed)
Q = 5

## Select which example to run

In [ ]:

def setInstance(instance):
    global i
    global f
    global o_verify
    
    i = ies[instance]
    f = fs[instance]
    o_verify = o_verifies[instance]

    print("Input activations")
    displayTensor(i.setColor("blue"))
    print("Filter Weights")
    displayTensor(f.setColor("green"))
    print("Output activations (expected)")
    displayTensor(o_verify)


select_instance = interactive(setInstance,
                              instance=["sparse", "dense"])

display(select_instance)




## Row Convolution - Input stationary


In [ ]:

o = Tensor(rank_ids=["M", "Q"])

canvas = createCanvas(i, f, o)

i_c = i.getRoot()
f_c = f.getRoot()
o_m = o.getRoot()

for c, (i_w, f_s) in i_c & f_c:
    for s, f_m in f_s:
       for m, (o_q, f_val) in  o_m << f_m:
            for q, (o_ref, i_val) in o_q << i_w.project(lambda w: w-s, interval=[0,Q-1]):
                o_ref += i_val * f_val
                addFrame(canvas, (c, q+s), (c, s, m), (m, q))

displayTensor(o)
displayCanvas(canvas)


## Check result

In [ ]:
o_verify == o

## Eyeriss V2 - Input

In [ ]:
# Same as above
eyeriss2_i = i.swapRanks()

print("Original input activations")
displayTensor(i)
print("Eyeriss v2 input activations - should be flattened")
displayTensor(eyeriss2_i)

## Eyeriss - V2 - No parallelism

Note: We assume that the number of inputs channels in the filter weights tensor (and packed into the input activations tensors) fit into the filter weight storage array, since this can be determined statically. We also assume that the number of output channels in the filter weights tensor also fit into the filter weight storage array and there are that same number of output activation partial sum buffers. 


In [ ]:

o = Tensor(rank_ids=["Q", "M"])

canvas = createCanvas(eyeriss2_i, f, o)

i_w = eyeriss2_i.getRoot()
f_c = f.getRoot()
o_q = o.getRoot()

for q in range(0, Q):
    o_m = o_q.getPayloadRef(q)

    for s in range(0,S):
        w = q+s
        i_c = i_w.getPayload(w)   # sliding window pattern

        for c, i_val in i_c:
            f_m = f_c.getPayload(c,s)

            for m, (o_ref, f_val) in o_m << f_m:
                o_ref += i_val * f_val
                addFrame(canvas, [(w, c)], [(c, s, m)], [(q, m)])

displayTensor(o)
displayCanvas(canvas)


## Check result

In [ ]:
eyeriss2_o_verify = o_verify.swapRanks()

displayTensor(eyeriss2_o_verify)
displayTensor(o)

eyeriss2_o_verify == o

## With parallelism

In [ ]:
o = Tensor(rank_ids=["Q", "M"])

canvas = createCanvas(eyeriss2_i, f, o)

i_w = eyeriss2_i.getRoot()
f_c = f.getRoot()
o_q = o.getRoot()

for q in range(0, Q):
    o_m = o_q.getPayloadRef(q)

    for s in range(0,S):
        w = q+s
        i_c = i_w.getPayload(w)

        for c, i_val in i_c:
            f_m = f_c.getPayload(c, s)
            f_m_split = f_m.splitEqual(2)
            
            for _, f_m in f_m_split: 
                fset = []
                oset = []
                for m, (o_ref, f_val) in o_m << f_m:   # parallel for
                    fset.append( (c, s, m) )
                    oset.append( (q, m) )
                    o_ref += i_val * f_val
                
                addFrame(canvas, [(w, c)], fset, oset)



displayTensor(o)
displayCanvas(canvas)


## Check result

In [ ]:

displayTensor(eyeriss2_o_verify)
displayTensor(o)

eyeriss2_o_verify == o

## Testing area

For running alternative algorithms

In [ ]:
displayTensor(o)
o.getRoot().uncompress()